In [1]:
import tensorflow as tf
import numpy as np
from observations import mnist

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(x_train, y_train), (x_test, y_test) = mnist("data/")

In [3]:
def load_mnist():
    (train_im, train_lab), (test_im, test_lab) = mnist('data/mnist')
    train_im_mean = np.mean(train_im, 0)
    train_im_std = np.std(train_im, 0)

    std_eps = 1e-7
    train_im = (train_im - train_im_mean) / (train_im_std + std_eps)
    test_im = (test_im - train_im_mean) / (train_im_std + std_eps)

    # train_im /= 255.0
    # test_im /= 255.0
    return (train_im, train_lab), (test_im, test_lab)

In [4]:
def one_hot(vector):
    out = np.zeros((10))
    out[vector-1] = 1
    return out

In [5]:
def conv_layer(data_in, size_in, size_out):
    with tf.name_scope("conv_"):
        nn_filter = tf.Variable(tf.truncated_normal((5,5,size_in, size_out),name="Weights"))
        bias = tf.Variable(tf.constant(0.1, shape=[size_out]), name="bias")
        conv_layer = tf.nn.conv2d(data_in, nn_filter, [1,1,1,1], "SAME")
        act = tf.nn.relu(conv_layer + bias)
        return tf.nn.max_pool(act,[1,2,2,1],strides=[1,1,1,1],padding="SAME",name="pooling")

In [6]:
def fully_connected(layer_in, size_in, size_out ):
    with tf.name_scope("fc_"):
        weights = tf.Variable(tf.truncated_normal((size_in, size_out)), name="Weights")
        bias = tf.Variable(tf.constant(0.1, shape=[size_out]), name="bias")
        act = tf.matmul(layer_in, weights) + bias
        return act

In [ ]:
def mnist_model(learning_rate):
    
    (x_train, y_train), _ = load_mnist()
    y_train = [one_hot(y) for y in y_train]
    sess = tf.Session()
    
    batch_size = 100
    x = tf.placeholder(tf.float32, shape=[None, 784], name="X")
    x_image = tf.reshape(x, [-1,28,28,1])
    y = tf.placeholder(tf.float32, shape=[None, 10], name="Y")
    
    conv = conv_layer(x_image, 1, 100)
    flattened = tf.reshape(conv, [-1, 28*28*100])
    logits = fully_connected(flattened, 28*28*100, 10)
    
    xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    
    train_step= tf.train.AdamOptimizer(learning_rate).minimize(xent)
    corr_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(corr_prediction, tf.float32))
    
    sess.run(tf.global_variables_initializer())
    
    batches_per_epoch = x_train.shape[0] // batch_size
    for i in range(20):
        for j in range(batches_per_epoch):
            x_batch = x_train[j * batch_size: (j+1) * batch_size]
            y_batch = y_train[j * batch_size: (j+1) * batch_size]
            
            sess.run(train_step, feed_dict={x:x_batch, y:y_batch})


In [ ]:
mnist_model(0.3)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

